In [1]:
!make docker-image > /dev/null 2>&1

In [2]:
!CONTAINER_CMD="bash -lc 'make install-ycsb" make docker > /dev/null 2>&1

In [3]:
from pathlib import Path
import pexpect
import os
import time

""" Collector class has helper methods to interact with kermit"""
class Collector: 
    def __init__(self, config: Path):
        self.env = os.environ.copy()
        self.env["INTERACTIVE"] = "it"
        self.env["CONTAINER_CMD"] = f"bash -lc 'KERNMLOPS_CONFIG_FILE={config} make collect-data'"
        self.collect_process : pexpect.spawn | None = None

    def start_collection(self, logfile=None):
        self.collect_process = pexpect.spawn("make docker", env=self.env, timeout=None, logfile=logfile)
        self.collect_process.expect_exact(["Started benchmark"])

    def _after_run_generate_file_data() -> dict[str, list[Path]]:
        start_path : Path = Path("./data")
        list_of_collect_id_dirs = start_path.glob("*/*/*")
        latest_collect_id = max(list_of_collect_id_dirs, key=os.path.getctime)
        list_of_files = latest_collect_id.glob("*.*.parquet")
        output = {}
        for f in list_of_files:
            index = str(f).removeprefix(str(f.parent) + "/").split(".")[0]
            if index not in output.keys():
                output[index] = []
            output[index].append(f)
        return output
        
    def wait(self) -> int:
        if self.collect_process is None:
            return
        self.collect_process.expect([pexpect.EOF])
        self.collect_process.wait()
        return Collector._after_run_generate_file_data()
        
    def stop_collection(self):
        if self.collect_process is None:
            return
        self.collect_process.sendline("END")
        return self.wait()

There are two ways to run kermit:
- With the raw config
- With a pre-programmed benchmark config

In [4]:
# Simple iperf3 Test - Docker Friendly Version

import subprocess
import time

# First, let's manually test if iperf3 works
print("Testing iperf3 installation...")

# Install iperf3 if needed
install_result = subprocess.run(["which", "iperf3"], capture_output=True)
if install_result.returncode != 0:
    print("Installing iperf3...")
    subprocess.run(["apt-get", "update"], capture_output=True)
    subprocess.run(["apt-get", "install", "-y", "iperf3"], capture_output=True)
    print("iperf3 installed!")
else:
    print("iperf3 is already installed")

# Kill any existing iperf3 processes
subprocess.run(["pkill", "-f", "iperf3"], capture_output=True)
time.sleep(1)

# Start iperf3 server manually
print("\nStarting iperf3 server on port 5555...")
server = subprocess.Popen(
    ["iperf3", "-s", "-p", "5555"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for server
time.sleep(3)

# Test connection
print("Testing server connection...")
test = subprocess.run(
    ["iperf3", "-c", "127.0.0.1", "-p", "5555", "-t", "1"],
    capture_output=True,
    text=True
)

if test.returncode == 0:
    print("✓ iperf3 is working!")
    
    # Now run actual benchmark with kernmlops
    print("\nRunning benchmark with kernmlops...")
    
    # Use minimal config for Docker
    collect = Collector("./config/iperf_docker.yaml")
    
    try:
        collect.start_collection(None)
        data = collect.wait()
        
        import polars as pl
        tcp_df = pl.read_parquet(data["tcp_v4_rcv"][0])
        print(f"\n✓ Success! Captured {len(tcp_df):,} TCP events")
        
    except Exception as e:
        print(f"\n✗ Benchmark failed: {e}")
        print("\nTry using the alternative method below...")
        
else:
    print("✗ iperf3 server test failed")
    print(f"Error: {test.stderr}")

# Cleanup
server.terminate()
subprocess.run(["pkill", "-f", "iperf3"], capture_output=True)


Testing iperf3 installation...
iperf3 is already installed

Starting iperf3 server on port 5555...
Testing server connection...
✓ iperf3 is working!

Running benchmark with kernmlops...

✓ Success! Captured 6,346 TCP events


CompletedProcess(args=['pkill', '-f', 'iperf3'], returncode=0, stdout=b'', stderr=b'')

In [5]:
# Analyze by process
print(tcp_df.group_by("comm").count().sort("count", descending=True))
iperf_client = tcp_df.filter(pl.col("comm").str.contains("iperf3").and_(~pl.col("comm").str.contains("-s")))

print(f"\nProcess breakdown:")
print(f"- iperf3 client: {len(iperf_client)} events")

# Analyze port 5555 traffic (iperf3 default port)
port_5555 = tcp_df.filter((pl.col("dport") == 5555) | (pl.col("sport") == 5555))
print(f"- Port 5555 traffic: {len(port_5555)} events")

# Branch distribution
print("\nTCP state distribution:")
branch_dist = tcp_df.group_by("branch_name").agg([
    pl.count().alias("count"),
    (pl.count() / len(tcp_df) * 100).alias("percentage")
]).sort("count", descending=True)

for row in branch_dist.head(10).iter_rows():
    print(f"- {row[0]}: {row[1]:,} events ({row[2]:.1f}%)")

# Connection analysis
new_connections = tcp_df.filter(pl.col("branch_name") == "new_syn_recv")
print(f"\nNew TCP connections: {len(new_connections)}")
print(f"Connections per stream: ~{len(new_connections) / 4:.0f}")  # 4 parallel streams

# Drop analysis
drops = tcp_df.filter(pl.col("drop_reason") > 0)
if len(drops) > 0:
    print(f"\nDropped packets: {len(drops)}")
    drop_dist = drops.group_by("drop_reason_name").len()
    for row in drop_dist.iter_rows():
        print(f"- {row[0]}: {row[1]} drops")
else:
    print("\nNo dropped packets detected - excellent!")

# Show branch distribution
print("iperf_client group by branch_name")
print(iperf_client.group_by("branch_name").count().sort("count", descending=True))

# Show drop reasons
drops = iperf_client.filter(pl.col("drop_reason") > 0)
print(drops.group_by("drop_reason_name").count())

shape: (8, 2)
┌─────────────────┬───────┐
│ comm            ┆ count │
│ ---             ┆ ---   │
│ str             ┆ u32   │
╞═════════════════╪═══════╡
│ iperf3          ┆ 2777  │
│ jupyter-noteboo ┆ 1356  │
│ sshd            ┆ 1151  │
│ code-2901c5ac6d ┆ 583   │
│ node            ┆ 465   │
│ swapper/8       ┆ 8     │
│ swapper/1       ┆ 3     │
│ swapper/0       ┆ 3     │
└─────────────────┴───────┘

Process breakdown:
- iperf3 client: 2777 events
- Port 5555 traffic: 799 events

TCP state distribution:
- entry: 2,003 events (31.6%)
- time_wait: 2,002 events (31.5%)
- new_syn_recv: 2,002 events (31.5%)
- socket_busy: 334 events (5.3%)
- listen_state: 4 events (0.1%)
- no_socket: 1 events (0.0%)

New TCP connections: 2002
Connections per stream: ~500

Dropped packets: 1
- no_socket: 1 drops
iperf_client group by branch_name
shape: (5, 2)
┌──────────────┬───────┐
│ branch_name  ┆ count │
│ ---          ┆ ---   │
│ str          ┆ u32   │
╞══════════════╪═══════╡
│ new_syn_recv ┆ 917   

/tmp/ipykernel_84880/4123034876.py:2: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  print(tcp_df.group_by("comm").count().sort("count", descending=True))
/tmp/ipykernel_84880/4123034876.py:15: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("count"),
/tmp/ipykernel_84880/4123034876.py:16: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  (pl.count() / len(tcp_df) * 100).alias("percentage")
/tmp/ipykernel_84880/4123034876.py:39: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  print(iperf_client.group_by("branch_name").count().sort("count", descending=True))
/tmp/ipykernel_84880/4123034876.py:43: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  print(drops.group_by("drop_reason_name").count())


In [15]:
import subprocess
import sys


# New TCP Collector
collect = Collector("./config/raw_overrides.yaml")
collect.start_collection()

# Generate some TCP traffic
!curl https://www.google.com > /dev/null
!nc -l 8080 &  # Listen on port 8080
!echo "test" | nc localhost 8080 
!curl https://www.google.com > /dev/null
!curl https://www.google.com > /dev/null
!curl https://www.google.com > /dev/null
!curl https://www.google.com > /dev/null

data = collect.stop_collection()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17579    0 17579    0     0   120k      0 --:--:-- --:--:-- --:--:--  120k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17578    0 17578    0     0   139k      0 --:--:-- --:--:-- --:--:--  139k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17618    0 17618    0     0   134k      0 --:--:-- --:--:-- --:--:--  135k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17463    0 17463    0     0   132k      0 --:--:-- --:--:-- --:--:--  133k
  % Total    % Received % Xferd  Average Speed   Tim

In [16]:
# Analyze TCP branches
import polars as pl
tcp_df = pl.read_parquet(data["tcp_v4_rcv"][0])

print(tcp_df.group_by("comm").count().sort("count", descending=True))

curl = tcp_df.filter((pl.col("dport") == 8080) | (pl.col("sport") == 8080))

# Show branch distribution
print(curl.group_by("branch_name").count().sort("count", descending=True))

# Show drop reasons
drops = curl.filter(pl.col("drop_reason") > 0)
print(drops.group_by("drop_reason_name").count())

shape: (14, 2)
┌─────────────────┬───────┐
│ comm            ┆ count │
│ ---             ┆ ---   │
│ str             ┆ u32   │
╞═════════════════╪═══════╡
│ ZMQbg/IO/0      ┆ 140   │
│ code-2901c5ac6d ┆ 115   │
│ jupyter-noteboo ┆ 89    │
│ swapper/13      ┆ 63    │
│ swapper/6       ┆ 57    │
│ …               ┆ …     │
│ node            ┆ 32    │
│ sshd            ┆ 12    │
│ nc              ┆ 6     │
│ swapper/3       ┆ 3     │
│ swapper/11      ┆ 2     │
└─────────────────┴───────┘
shape: (1, 2)
┌─────────────┬───────┐
│ branch_name ┆ count │
│ ---         ┆ ---   │
│ str         ┆ u32   │
╞═════════════╪═══════╡
│ entry       ┆ 2     │
└─────────────┴───────┘
shape: (0, 2)
┌──────────────────┬───────┐
│ drop_reason_name ┆ count │
│ ---              ┆ ---   │
│ str              ┆ u32   │
╞══════════════════╪═══════╡
└──────────────────┴───────┘


/tmp/ipykernel_84880/735243307.py:5: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  print(tcp_df.group_by("comm").count().sort("count", descending=True))
/tmp/ipykernel_84880/735243307.py:10: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  print(curl.group_by("branch_name").count().sort("count", descending=True))
/tmp/ipykernel_84880/735243307.py:14: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  print(drops.group_by("drop_reason_name").count())


In [17]:
tcp_df

cpu,pid,tgid,ts_uptime_us,branch_type,branch_name,drop_reason,drop_reason_name,saddr,daddr,sport,dport,comm,collection_id
i64,i64,i64,i64,i64,str,i64,str,str,str,i64,i64,str,str
0,8170,8170,194915800585,0,"""entry""",0,"""none""","""1.0.0.127""","""1.0.0.127""",8888,47572,"""jupyter-noteboo""","""1df03379-a653-41cd-9eed-c25331…"
0,8170,8170,194915800592,3,"""time_wait""",0,"""none""","""0.0.0.0""","""0.0.0.0""",0,0,"""jupyter-noteboo""","""1df03379-a653-41cd-9eed-c25331…"
0,8170,8170,194915800594,8,"""new_syn_recv""",0,"""none""","""0.0.0.0""","""0.0.0.0""",0,0,"""jupyter-noteboo""","""1df03379-a653-41cd-9eed-c25331…"
0,8170,8170,194915800610,0,"""entry""",0,"""none""","""1.0.0.127""","""1.0.0.127""",47572,8888,"""jupyter-noteboo""","""1df03379-a653-41cd-9eed-c25331…"
0,8170,8170,194915800612,3,"""time_wait""",0,"""none""","""0.0.0.0""","""0.0.0.0""",0,0,"""jupyter-noteboo""","""1df03379-a653-41cd-9eed-c25331…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…
13,0,0,194917158974,3,"""time_wait""",0,"""none""","""0.0.0.0""","""0.0.0.0""",0,0,"""swapper/13""","""1df03379-a653-41cd-9eed-c25331…"
13,0,0,194917158978,8,"""new_syn_recv""",0,"""none""","""0.0.0.0""","""0.0.0.0""",0,0,"""swapper/13""","""1df03379-a653-41cd-9eed-c25331…"
13,0,0,194917249502,0,"""entry""",0,"""none""","""191.109.130.158""","""31.217.110.128""",49274,22,"""swapper/13""","""1df03379-a653-41cd-9eed-c25331…"


In [8]:
collect = Collector("./config/redis_never.yaml")
collect.start_collection(None)
data = collect.stop_collection()

In [9]:
# Analyze TCP branches
import polars as pl
tcp_df = pl.read_parquet(data["tcp_v4_rcv"][0])

print(tcp_df.group_by("comm").count().sort("count", descending=True))

redis = tcp_df.filter(pl.col("comm").str.contains("redis-server"))

# Show branch distribution
print(redis.group_by("branch_name").count().sort("count", descending=True))

# Show drop reasons
drops = redis.filter(pl.col("drop_reason") > 0)
print(drops.group_by("drop_reason_name").count())

shape: (30, 2)
┌─────────────────┬───────┐
│ comm            ┆ count │
│ ---             ┆ ---   │
│ str             ┆ u32   │
╞═════════════════╪═══════╡
│ Thread-2        ┆ 10040 │
│ redis-server    ┆ 8311  │
│ jupyter-noteboo ┆ 5622  │
│ swapper/5       ┆ 4038  │
│ sshd            ┆ 3988  │
│ …               ┆ …     │
│ G1 Conc#2       ┆ 3     │
│ kworker/10:0    ┆ 3     │
│ ksoftirqd/10    ┆ 3     │
│ ksoftirqd/9     ┆ 3     │
│ rcu_preempt     ┆ 3     │
└─────────────────┴───────┘
shape: (4, 2)
┌──────────────┬───────┐
│ branch_name  ┆ count │
│ ---          ┆ ---   │
│ str          ┆ u32   │
╞══════════════╪═══════╡
│ entry        ┆ 2768  │
│ time_wait    ┆ 2768  │
│ new_syn_recv ┆ 2768  │
│ socket_busy  ┆ 7     │
└──────────────┴───────┘
shape: (0, 2)
┌──────────────────┬───────┐
│ drop_reason_name ┆ count │
│ ---              ┆ ---   │
│ str              ┆ u32   │
╞══════════════════╪═══════╡
└──────────────────┴───────┘


/tmp/ipykernel_84880/466112565.py:5: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  print(tcp_df.group_by("comm").count().sort("count", descending=True))
/tmp/ipykernel_84880/466112565.py:10: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  print(redis.group_by("branch_name").count().sort("count", descending=True))
/tmp/ipykernel_84880/466112565.py:14: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  print(drops.group_by("drop_reason_name").count())


In [10]:
# Create collector with XSBench configuration
collect = Collector("./config/xsbench.yaml")

# Start collection and run XSBench
print("Starting collection with XSBench workload...")
collect.start_collection(None)

# Wait for XSBench to complete
print("Running XSBench benchmark...")
data = collect.stop_collection()




Starting collection with XSBench workload...
Running XSBench benchmark...


In [11]:
# Analyze TCP traffic generated by XSBench
print("\nAnalyzing TCP traffic from XSBench:")
tcp_df = pl.read_parquet(data["tcp_v4_rcv"][0])

# Show branch distribution
print("\nTCP branch distribution:")
print(tcp_df.group_by("branch_name").len().sort("len", descending=True))

# Show drop reasons if any
drops = tcp_df.filter(pl.col("drop_reason") > 0)
if len(drops) > 0:
    print("\nDropped packets:")
    print(drops.group_by("drop_reason_name").len())
else:
    print("\nNo dropped packets detected")

# Show process-specific TCP activity
print("\nTCP activity by process:")
process_tcp = tcp_df.group_by("comm").len().sort("len", descending=True).head(10)
print(process_tcp)

# Check for XSBench-specific activity
xsbench_traffic = tcp_df.filter(pl.col("comm").str.contains("XSBench"))
if len(xsbench_traffic) > 0:
    print(f"\nXSBench generated {len(xsbench_traffic)} TCP events")
else:
    print("\nNo direct TCP traffic from XSBench process detected")


Analyzing TCP traffic from XSBench:

TCP branch distribution:
shape: (4, 2)
┌──────────────┬─────┐
│ branch_name  ┆ len │
│ ---          ┆ --- │
│ str          ┆ u32 │
╞══════════════╪═════╡
│ entry        ┆ 12  │
│ time_wait    ┆ 12  │
│ new_syn_recv ┆ 12  │
│ socket_busy  ┆ 5   │
└──────────────┴─────┘

No dropped packets detected

TCP activity by process:
shape: (4, 2)
┌─────────────────┬─────┐
│ comm            ┆ len │
│ ---             ┆ --- │
│ str             ┆ u32 │
╞═════════════════╪═════╡
│ ZMQbg/IO/0      ┆ 14  │
│ code-2901c5ac6d ┆ 14  │
│ jupyter-noteboo ┆ 7   │
│ swapper/13      ┆ 6   │
└─────────────────┴─────┘

No direct TCP traffic from XSBench process detected
